In [6]:
import duckdb

con = duckdb.connect()

con.execute("""
CREATE TABLE learner_activity AS
SELECT * FROM read_csv_auto('../data/raw/learner_activity_raw.csv')
""")


In [7]:
con.execute("""
SELECT COUNT(*) AS total_rows 
FROM learner_activity
""").fetchall()


[(9,)]

In [4]:
%pip install duckdb


   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/12.3 MB 8.4 MB/s eta 0:00:02
   ---------- ----------------------------- 3.1/12.3 MB 7.9 MB/s eta 0:00:02
   ----------------- ---------------------- 5.2/12.3 MB 8.9 MB/s eta 0:00:01
   --------------------- ------------------ 6.6/12.3 MB 8.4 MB/s eta 0:00:01
   ------------------------- -------------- 7.9/12.3 MB 8.0 MB/s eta 0:00:01
   ---------------------------- ----------- 8.7/12.3 MB 8.0 MB/s eta 0:00:01
   ------------------------------- -------- 9.7/12.3 MB 6.9 MB/s eta 0:00:01
   -------------------------------------- - 11.8/12.3 MB 7.2 MB/s eta 0:00:01
   ---------------------------------------- 12.3/12.3 MB 7.2 MB/s  0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
con.execute("""
SELECT COUNT(*) AS total_rows
FROM learner_activity
""").fetchall()


[(9,)]

In [9]:
con.execute("""
SELECT COUNT(*) AS invalid_video_progress
FROM learner_activity
WHERE video_progress_pct > 100
   OR video_progress_pct < 0
""").fetchall()


[(1,)]

In [12]:
import pandas as pd

df = pd.read_csv('../data/raw/learner_activity_raw.csv')
df.head()


,user_id,course_id,enrollment_date,module_name,video_progress_pct,time_spent_minutes,assessment_attempts,assessment_score,completion_status,feedback_rating,device_type,access_time
0,U001,DS101,01-06-2024,Intro to Data,100.0,45,1.0,85,Y,5.0,Mobile,10:30 AM
1,U001,DS101,2024/06/02,SQL Basics,55.0,30,2.0,60,Yes,4.0,mobile,22:15
2,U001,DS101,06-03-2024,Python Basics,120.0,0,NaN,,N,NaN,Desktop,9 PM
3,U002,DS101,2024-06-01,Intro to Data,80.0,40,1.0,70,Y,4.0,Laptop,11:00
4,U002,DS101,2024-06-04,SQL Basics,NaN,25,3.0,45,No,3.0,desktop,14:20


In [13]:
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              9 non-null      object 
 1   course_id            9 non-null      object 
 2   enrollment_date      9 non-null      object 
 3   module_name          9 non-null      object 
 4   video_progress_pct   8 non-null      float64
 5   time_spent_minutes   9 non-null      int64  
 6   assessment_attempts  7 non-null      float64
 7   assessment_score     9 non-null      object 
 8   completion_status    9 non-null      object 
 9   feedback_rating      7 non-null      float64
 10  device_type          9 non-null      object 
 11  access_time          9 non-null      object 
dtypes: float64(3), int64(1), object(8)
memory usage: 996.0+ bytes


In [14]:
df['enrollment_date'] = pd.to_datetime(
    df['enrollment_date'],
    errors='coerce'
)


In [15]:
df['video_progress_pct'] = pd.to_numeric(
    df['video_progress_pct'],
    errors='coerce'
)

df['video_progress_pct'] = df['video_progress_pct'].clip(0, 100)



In [16]:
df['time_spent_minutes'] = pd.to_numeric(
    df['time_spent_minutes'],
    errors='coerce'
)


In [17]:
df['assessment_attempts'] = pd.to_numeric(
    df['assessment_attempts'],
    errors='coerce'
).fillna(0).astype(int)

df['assessment_score'] = pd.to_numeric(
    df['assessment_score'],
    errors='coerce'
).fillna(0)


In [18]:
df['completion_status'] = (
    df['completion_status']
    .astype(str)
    .str.strip()
    .str.lower()
    .map({
        'y': 'Yes',
        'yes': 'Yes',
        'n': 'No',
        'no': 'No'
    })
)


In [19]:
df['device_type'] = (
    df['device_type']
    .astype(str)
    .str.strip()
    .str.lower()
)


In [20]:
df['feedback_rating'] = pd.to_numeric(
    df['feedback_rating'],
    errors='coerce'
)


In [21]:
df.isna().sum()


user_id                0
course_id              0
enrollment_date        7
module_name            0
video_progress_pct     1
time_spent_minutes     0
assessment_attempts    0
assessment_score       0
completion_status      0
feedback_rating        2
device_type            0
access_time            0
dtype: int64

In [24]:
import os
os.getcwd()


'c:\\Users\\Arthi\\Learner-Journey-Funnel-Drop-off-Analysis\\notebooks'

In [25]:
import os

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
CLEANED_PATH = os.path.join(
    BASE_DIR, "data", "cleaned", "learner_activity_cleaned.csv"
)

CLEANED_PATH


'c:\\Users\\Arthi\\Learner-Journey-Funnel-Drop-off-Analysis\\data\\cleaned\\learner_activity_cleaned.csv'

In [27]:
df.to_csv(CLEANED_PATH, index=False)
